### Database Exploration

**The objectives of this notebook are to:** 

1. Explore the database generated by the generate_database.py and the update_database.py script. Then an attempt will be made to increase the number of succesful hits and add these to the database.
2. Check the images obtained from the google image search (get_plant_images.py) and see if any of these need to be manually added or changed to a more appropriate image. If so the current versions will be overwritten and the attirubtion column in the SQL database will need to be updated. 

**Summary of this notebook: Part 1**
 
- It's clear that many of the plant latin names that were extracted from Plantagen's website were not found with the google search API at the site: https://missouribotanicalgarden.org/. 
- This is not because of an issue with the script per say, just simply there were many not available in the database.
- A smaller subsection could have been found at the database but contained Swedish words in their title (such as "gruppen" - "the group").
- These and others issues have been manually taken care of to help expand the database. 

<ins>Originally the database had:</ins>
| Searches Made  | Links found | Links Not found |
| ---- | ---- |----|
| 368 | 126  | 242|

<ins>Now (after manual intervention) the database has:</ins>

| Searches Made  | Links found | Links Not found |
| ---- | ---- |----|
| N/A (368) | 147  | 242|


A lot more work could be put into increasing the database size, but as this is a fun side project for me to learn from, I will not put in further effort towards this. 



**Summary of this notebook: Part 2**

- Several plant images needed updating and to make life easier in the plotly app, they were resized (to all have a 1:1 aspect ratio) and some were compressed so as to take up less space. 
- Now all paths are working correctly on the app and images are of reasonable quality so I am happy with this notebook.


**Note that notebook was re-ran from start to finish so some of the cellblock outputs wont match what was written nearby in markdown above (as the manual additions were already added at the time of re-running).**

### Part 1: Check URLs found and see if more can be added.  

In [2]:
import sqlite3
import pandas as pd

In [3]:
# read in database, extract all plants and summarise result
DATABASE_LOC = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\PyForFun\House_Plant_Recommender\Database\house_plants.db"

conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.execute("""SELECT * FROM 'hyperlinks'""")

plants_found, plants_not_found = [], []
found, not_found = 0, 0

for row in c.fetchall():
    
    if row[1] == "no link found":
        not_found += 1
        plants_not_found.append(row)
    else:
        found += 1
        plants_found.append(row)
print(f"Total number of links now searched: {(found+not_found)}")
print(f"Number of links found: {found}")
print(f"Number of links not found: {not_found}")

c.close()

Total number of links now searched: 389
Number of links found: 147
Number of links not found: 242


In [4]:
missing_plants = [name[0] for name in plants_not_found] 
missing_plants

['Sansevieria',
 'Anthurium Andraeanum-Gruppen',
 'Myrsine africana',
 'Strelitzia nicolai',
 'Pelargonium (Peltatum-Zonale-Gruppen)',
 'Crassula coccinea',
 'Echeveria',
 'Parahemionitis cordata',
 'Alocasia gageana',
 'Nepenthes',
 'Bouvardia x domestica',
 'Sophora prostrata',
 'Cymbidium',
 'Tulipa gesneriana',
 'Wallisia cyanea',
 'Primula obconica',
 'Disocactus anguliger',
 'Cissus striata',
 'Philodendron erubescens',
 'Crocus x hybridus',
 'Haworthia attenuata',
 'Calathea warscewiczii',
 'Goeppertia rufibarba',
 'Pilea depressa',
 'Muscari botryoides',
 'Cissus rotundifolia',
 'x Oncidopsis grex',
 'Zamioculcas zamiifolia',
 'Sarracenia',
 'Muehlenbeckia axillaris',
 'Centella asiatica',
 'Philodendron',
 'Kalanchoe delagoensis',
 'Vriesea',
 'Euphorbia tithymaloides',
 'Goeppertia ornata',
 'Araucaria cunninghamii',
 'Echeveria purpusorum',
 'Ficus deltoidea',
 'Oreocereus trollii',
 'Alocasia baginda',
 'Ornithogalum dubium',
 'Ficus cyathistipula',
 'Euphorbia fruticosa',


#### Some notes from analysing the list above

- Some names contain words in swedish e.g.: 'Kalanchoe (Blossfeldiana-Gruppen)' and  'Pelargonium Peltatum-Gruppen', so these obviously would not match. 
- Some names contain abbreviations e.g: 'Ficus americana ssp. guianensis'. May cause match issues. 
- I did take a random selection and run these names through both google (to confirm real name) and through the https://missouribotanicalgarden.org search API to confirm this plant is indeed not present in the database. So the automated method is working fine as far I can tell.  


#### On adding to the database
- Clearly, we can increase the list of plants (with links) in the databse. For example, 'Pelargonium Peltatum-Gruppen' gives no matches, but: 'Pelargonium peltatum' https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a535 does. 

- Whilst the ficus species 'Ficus cyathistipula' (and others) were not found, the missouribotanicalgarden.org database has results for many other ficus spcecies that are houseplants and can also be added to the database. 

- If desired, a lot of time could be invested into increasing the size of the database (including by adding other sources).


In [5]:
more_plants = [
    # Those that had Swedish words (now removed/translated) in their name:
    ("Anthurium andraeanum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=276219"),
    ("Pelargonium peltatum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a535"),
    ("Kalanchoe blossfeldiana", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=279373"),
    ("Narcissus 'Golden Dawn'", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=260291"),
    ("Rosa rubiginosa", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=286363"), 
    ("Musa acuminata", "http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282778"), 
    ("Begonia rex-cultorum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=242218"), 
    ("Dracaena fragrans 'Lemon Lime'", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=366918"),
    ("Dracaena fragrans", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282260"),  
    ("Dracaena marginata", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=b592"),  
    ("Fittonia albivenis", "http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=b601"),  
    ("Pelargonium × hortorum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a537"),  
    ("Hippeastrum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=264599&basic=Hippeastrum"),  
    ("Pelargonium scented-leaves", "http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a534"),  
    ("Pelargonium × hortorum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a537"),  
    ("Cyclamen persicum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a444"),  
    ("Capsicum annuum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=287148"),  
    # Those with acronyms.  
    ("Sarracenia rubra", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=286847"),  
    ("Sarracenia flava", "http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=b927"),  
    ("Pericallis × hybrida", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=263628"),  
    ("Begonia", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=263000"),  
    # Available ficus plants.  
    ("Ficus lyrata", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282753"),
    ("Ficus carica", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282762"),
    ("Ficus benjamina", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282745"),
    ("Ficus elastica", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=b597"),
    ("Ficus pumila", "http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=b599"),
    ("Ficus religiosa", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282754")
]

len(more_plants)

27

Before adding these to the database, I will need to do a duplicates check. I will check for duplicates in both the url and the name to increase the chance of catching anything. 
As the URL can have extra bits of text tagged on at the end, I have made sure that the "plants_to_add" members all have those bits removed and then I will check if their string is in any of the urls already in the database, that should work... 

In [6]:
# lists to check against. 
plant_names_check = [plant[0].lower() for plant in plants_found]
plant_urls_check = [plant[1] for plant in plants_found]

In [7]:
duplicate_found = []
plants_to_add = []
for new_plant in more_plants:
    # name check
    if ((new_plant[0].lower() in plant_names_check) and (new_plant[1] in plant_urls_check)):
       duplicate_found.append(new_plant)
    else:
        plants_to_add.append(new_plant)
print(f"Plants without duplicates: {len(plants_to_add)}")
print(f"Plants with duplicates: {len(duplicate_found)}")

# Note that the notebook was re-run from start to finish hence why it now says there are no duplicates. 

Plants without duplicates: 0
Plants with duplicates: 27


In [8]:
# Checking those with duplicates
for duplicate in duplicate_found:
    print(f"Searching for duplicates for plant: {duplicate[0]} \n")
    if duplicate[0] in plant_names_check:
        print(duplicate[0])

    if duplicate[1] in plant_urls_check:
        print(duplicate[1])

Searching for duplicates for plant: Anthurium andraeanum 

https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=276219
Searching for duplicates for plant: Pelargonium peltatum 

https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?kempercode=a535
Searching for duplicates for plant: Kalanchoe blossfeldiana 

https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=279373
Searching for duplicates for plant: Narcissus 'Golden Dawn' 

https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=260291
Searching for duplicates for plant: Rosa rubiginosa 

https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=286363
Searching for duplicates for plant: Musa acuminata 

http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=282778
Searching for duplicates for plant: Begonia rex-cultorum 

https://www.missouribotanicalgarden.org/PlantF

All duplicates have both a duplicated name and url, so I am happy they are true duplicates and need not be included.

In [9]:
# Now it is time to add these extra urls to the database. 
# Primary key is the latin name, so that will act as one final check. 
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.executemany("""INSERT OR IGNORE INTO hyperlinks VALUES (?,?)""", plants_to_add)
conn.commit()

In [10]:
# Now summarise the final form of the database. 
c.execute("""SELECT * FROM 'hyperlinks'""")

output = []
found, not_found = 0, 0

for row in c.fetchall():
    output.append(row)

    if row[1] == "no link found":
        not_found += 1
    else:
        found += 1

print(f"Total number of links now searched: {len(output)}")
print(f"Number of links found: {found}")
print(f"Number of links not found: {not_found}")

Total number of links now searched: 389
Number of links found: 147
Number of links not found: 242


In [11]:
c.close()

##### Have to update one of the manually added links as it was inputted incorrectly

In [12]:
# correct form: ("Hippeastrum", "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=264599&basic=Hippeastrum"),  
# Getting the old hyperlink below
for plant in plants_found:
    if plant[0] == "Hippeastrum":
        print(plant)

('Hippeastrum', 'https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=264599&basic=Hippeastrum')


In [13]:
# update 1 - now hyperlink works. 
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.execute("""
    UPDATE hyperlinks
    SET url = "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=264599&basic=Hippeastrum"
    WHERE url = "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails.aspx?taxonid=26459"
""")
conn.commit()
c.close()

In [14]:
# update 2 - remove brackets as causing issues saving paths for images. 
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.execute("""
    UPDATE hyperlinks
    SET plant_name = "Pelargonium scented-leaves"
    WHERE plant_name = "Pelargonium (scented-leaved group)"
""")

conn.commit()
c.close()

### Part 2: Check images and see if any need to be added or updated 

In [15]:
# read in database, extract all plants and summarise result
DATABASE_LOC = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\PyForFun\House_Plant_Recommender\Database\house_plants.db"

conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
image_df = pd.read_sql_query("SELECT * FROM 'plant_images'", conn)
c.close()

In [16]:
image_df["Website"].value_counts()

commons.wikimedia.org    122
www.flickr.com            13
en.wikipedia.org           3
pxhere.com                 2
www.pikist.com             2
www.maxpixel.net           1
flickr.com                 1
My own house plant         1
www.pxfuel.com             1
en.m.wikipedia.org         1
Name: Website, dtype: int64

In [17]:
# One missing plant:
image_df.loc[image_df["File_Path"] == "no image found"]  

,Plant_Name,File_Path,Website


Previously there was one plant not found, which was addressed in the cell block below. 

The Dr Westerlund is among these members, which I have at home, so I can add a photo of my own one. 

In [18]:
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.execute("""UPDATE 'plant_images' SET 
    File_Path = 'Database/images/Pelargonium_scented-leaves.jpeg', 
    Website = 'My own house plant'
    WHERE Plant_Name = 'Pelargonium scented-leaves'"""
)
conn.commit()
c.close()

Now will make a list of those plant images which from the automated search are not great (could be low quality, not actully a picture of the plant etc...):

In [19]:
# Files were saved with "_" used instead of spaces, so will convert them back to match the latin_name stored in the db. 
# To make life easy I will take any new picture from commons.wikimedia.org, so can update them all in the database
to_update = ["Alocasia_macrorrhizos", "Alocasia_zebrina" , "Anthurium_crassinervium", "Ardisia_crenata", 
    "Asplenium", "Begonia_masoniana", "Begonia_rex-cultorum", "Capsicum_annuum", "Cissus_alata", 
    "Dendrobium", "Elettaria_cardamomum", "Euphorbia_lactea" , "Euphorbia_tirucalli", "Ginkgo_biloba", 
    "Oncidium", "Nephrolepis_exaltata", "Platycerium_bifurcatum", "Punica_granatum", "Radermachera_sinica", 
    "Schefflera_arboricola", "Scilla_mischtschenkoana", "Woodsia_obtusa"]

In [20]:
image_update_list = []
for plant in to_update:
    image_update_list.append( ("commons.wikimedia.org", plant.replace("_", " ")) )
image_update_list[0:3]

[('commons.wikimedia.org', 'Alocasia macrorrhizos'),
 ('commons.wikimedia.org', 'Alocasia zebrina'),
 ('commons.wikimedia.org', 'Anthurium crassinervium')]

In [21]:
# Now update there url in the database. 
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
c.executemany("UPDATE plant_images SET Website=? WHERE Plant_Name=?", image_update_list)
conn.commit()
c.close()

## Extra Addition

In order to use images with the plotly dash app, they need to be stored in a folder called assets, which I have now created and moved the image files too.

Further, after compressing each image, their file name was autochanged to .jpg from .jpeg or .png.  

Now I need to update the file paths for each plant accordingly.

In [22]:
image_df["File_Path"] = image_df["File_Path"].str.replace("Database/images/", "assets/")
image_df["File_Path"] = image_df["File_Path"].str.replace(".jpeg", ".jpg")
image_df["File_Path"] = image_df["File_Path"].str.replace(".png", ".jpg")
image_df.head(3)

<ipython-input-22-4ab856c1370f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  image_df["File_Path"] = image_df["File_Path"].str.replace(".jpeg", ".jpg")
<ipython-input-22-4ab856c1370f>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  image_df["File_Path"] = image_df["File_Path"].str.replace(".png", ".jpg")


,Plant_Name,File_Path,Website
0,Aechmea,assets/Aechmea_resized.jpg,commons.wikimedia.org
1,Ardisia crenata,assets/Ardisia_crenata_resized.jpg,commons.wikimedia.org
2,Euphorbia milii,assets/Euphorbia_milii_resized.jpg,commons.wikimedia.org


### Final update is to revert pictures which were resized to png. 

In [23]:
image_df["File_Path"] = image_df["File_Path"].str.replace("_resized.jpg", "_resized.png")
image_df.head(3)

<ipython-input-23-e0819f6cdb4d>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  image_df["File_Path"] = image_df["File_Path"].str.replace("_resized.jpg", "_resized.png")


,Plant_Name,File_Path,Website
0,Aechmea,assets/Aechmea_resized.png,commons.wikimedia.org
1,Ardisia crenata,assets/Ardisia_crenata_resized.png,commons.wikimedia.org
2,Euphorbia milii,assets/Euphorbia_milii_resized.png,commons.wikimedia.org


In [26]:
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()

c.execute("""DROP TABLE IF EXISTS plant_images""")

image_df.to_sql("plant_images", con=conn, if_exists="append", index=False)

test_saved_df = pd.read_sql_query("SELECT * FROM plant_images", conn)

conn.commit()
c.close()

In [27]:
test_saved_df

,Plant_Name,File_Path,Website
0,Aechmea,assets/Aechmea_resized.png,commons.wikimedia.org
1,Ardisia crenata,assets/Ardisia_crenata_resized.png,commons.wikimedia.org
2,Euphorbia milii,assets/Euphorbia_milii_resized.png,commons.wikimedia.org
3,Ficus elastica,assets/Ficus_elastica_resized.png,commons.wikimedia.org
4,Woodsia obtusa,assets/Woodsia_obtusa_resized.png,commons.wikimedia.org
...,...,...,...
142,Begonia,assets/Begonia_resized.png,commons.wikimedia.org
143,Ficus carica,assets/Ficus_carica_resized.png,commons.wikimedia.org
144,Ficus pumila,assets/Ficus_pumila_resized.png,commons.wikimedia.org
145,Ficus religiosa,assets/Ficus_religiosa_resized.png,commons.wikimedia.org
